In [1]:
import timeit
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory

image_size = 28
num_labels = 10
num_channels = 1

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape, type(train_dataset), type(train_labels))
print('Validation set', valid_dataset.shape, valid_labels.shape, type(valid_dataset), type(valid_labels))
print('Test set', test_dataset.shape, test_labels.shape, type(test_dataset), type(test_labels))
print('train_labels.shape[0]', train_labels.shape[0])

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print("defined accuracy function")

('Training set', (200000, 28, 28, 1), (200000, 10), <type 'numpy.ndarray'>, <type 'numpy.ndarray'>)
('Validation set', (10000, 28, 28, 1), (10000, 10), <type 'numpy.ndarray'>, <type 'numpy.ndarray'>)
('Test set', (18724, 28, 28, 1), (18724, 10), <type 'numpy.ndarray'>, <type 'numpy.ndarray'>)
('train_labels.shape[0]', 200000)
defined accuracy function


In [2]:
batch_size = 24
patch_size = 3
depth = 18
num_hidden = 128
L2_reg_const = 0.5e-3

graph = tf.Graph()

with graph.as_default():

  dropout_p = tf.placeholder(tf.float32)
  
  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  layer0_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer0_biases = tf.Variable(tf.zeros([depth]))

  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([12*12* depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  L2_reg = (tf.nn.l2_loss(layer0_weights) + tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights)
            + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights)  + tf.nn.l2_loss(layer5_weights))  

  # Model.
  def model(data, dropout_p, verbose = False):
    
    conv0 = tf.nn.relu((tf.nn.conv2d(data,layer0_weights,[1, 1, 1, 1],padding='SAME'))+layer0_biases)
    
    conv1 = tf.nn.relu((tf.nn.conv2d(conv0,layer1_weights,[1, 1, 1, 1],padding='VALID'))+layer1_biases)
    
    conv2 = tf.nn.relu((tf.nn.conv2d(conv1, layer2_weights, [1, 2, 2, 1], padding='VALID')) + layer2_biases)
    
    shape = conv2.get_shape().as_list() 
       
    reshape2 = tf.reshape(conv2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden3 = tf.nn.relu(tf.matmul(reshape2, layer3_weights) + layer3_biases)
    
    layer_dropout3 = tf.nn.dropout(hidden3, dropout_p)
    
    hidden4 = tf.nn.relu(tf.matmul(layer_dropout3, layer4_weights) + layer4_biases)
    
    final_output = tf.matmul(hidden4, layer5_weights) + layer5_biases
    
    if verbose == True:
        print 'data shape', data.get_shape().as_list() # [16, 28, 28, 1]
        print 'conv0', conv0.get_shape().as_list()
        print 'conv1', conv1.get_shape().as_list()
        print 'conv2', conv2.get_shape().as_list()
    
    return final_output
  
  # Training computation.
  logits = model(tf_train_dataset, dropout_p)
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2_reg_const*L2_reg )
    
  # Optimizer.

  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate,global_step,100,0.99,staircase=True)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(model(tf_train_dataset, dropout_p, verbose = True))
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, dropout_p = 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, dropout_p = 1.0))

print 'Nodes created over computation graph'
print 'notice how in this architecture, learning seems to take longer to show increase in accuracy'
print 'at step 18000 and LR 0.018 finally the accuracy jumps from 10.4% which is almost random to  '
print ' 73.9% which is a 64% change in validation accuracy'

data shape [24, 28, 28, 1]
conv0 [24, 28, 28, 18]
conv1 [24, 26, 26, 18]
conv2 [24, 12, 12, 18]
Nodes created over computation graph
notice how in this architecture, learning seems to take longer to show increase in accuracy
at step 18000 and LR 0.018 finally the accuracy jumps from 10.4% which is almost random to  
 73.9% which is a 64% change in validation accuracy


In [ ]:
num_steps = 2400

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  start_time = timeit.default_timer()

  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict_train = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_p: 0.5}
        
    _, training_loss, LR = session.run([optimizer, loss, learning_rate], feed_dict=feed_dict_train)
    
    feed_train_predict = {tf_train_dataset : batch_data, dropout_p: 1}
        
    batch_predictions = session.run(train_prediction, feed_dict=feed_train_predict)
    
    if (step % 100 == 0): 
       print('learning rate',LR)
       print("Minibatch training set loss at step %d: %f" % (step, training_loss))
       print("Minibatch training set accuracy: %.1f%%" % accuracy(batch_predictions, batch_labels))
       print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
  end_time = timeit.default_timer()
  #feed_test_predict = {tf_test_dataset : test_dataset, dropout_p: 1}
  #test_predictions = session.run(test_prediction, feed_dict=feed_test_predict)
  print("Finished: Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print('ran for %.2fm' % ((end_time - start_time) / 60.))

Initialized
('learning rate', 0.1)
Minibatch training set loss at step 0: 8.408677
Minibatch training set accuracy: 12.5%
Validation accuracy: 10.1%
('learning rate', 0.098999999)
Minibatch training set loss at step 100: 2.379555
Minibatch training set accuracy: 75.0%
Validation accuracy: 63.2%
('learning rate', 0.098010004)
Minibatch training set loss at step 200: 1.694413
Minibatch training set accuracy: 75.0%
Validation accuracy: 73.5%
('learning rate', 0.097029902)
Minibatch training set loss at step 300: 1.737690
Minibatch training set accuracy: 79.2%
Validation accuracy: 76.0%
('learning rate', 0.096059605)
Minibatch training set loss at step 400: 1.866930
Minibatch training set accuracy: 66.7%
